# Setup Your Analysis Environment

1) Choose a directory to house your project in: 
```
.../<project-directory>
```


2) Create and activate a fresh Python3 virtual environment there: 
```
$ cd .../<project-directory>
$ python -m virtualenv env #Package tested on Python 3.6.8
$ source env/bin/activate
```

3) Download the `cosmicfish` package from Git: 
```
$ git clone git@github.com:ndeporzio/cosmicfish.git
```

4) Install the `cosmicfish` package. Note that its dependencies will install automatically.
```
$ cd cosmicfish
$ pip install . 
```

5) Launch Jupyter and open `tutorial.ipynb` notebook using Jupyter browser
```
$ jupyter notebook
```

6) Create a data folder where the analysis can store spectrum data. This can be anywhere you'd like - you'll specify the path below. 
```
$ mkdir <project-directory>/data
```

7) Install and build CLASS (if you don't already have a build). Note the `cosmicfish` package includes a method for downloading and installing CLASS for you. 
```
$ python 
>>> import cosmicfish
>>> cosmicfish.install_class('<project-directory>/class')
>>> exit()
```

# Setup Fiducial Cosmology

Import the analysis package.

In [ ]:
import cosmicfish as cf 

Import relevant python packages... 

In [ ]:
#Import relevant packages
import os
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

Other setup steps... 

In [ ]:
#Instruct pyplot to use seaborn 
sns.set()

Specify the paths from the setup of your analysis environment above.  

In [ ]:
#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/test/")
datastore = cf.correct_path("/Volumes/SSD01/data/")
classpath = os.path.join(projectdir, "class")

Specify the values of your fiducial cosmology and other physical constants.

In [ ]:
#Setup fiducial cosmology, note change to N_ncdm... 
fid = {"A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "N_ncdm" : 1, #Note the change! 
        "m_ncdm" : 0.1, #Comment out for N_ncdm=0
        "T_ncdm" : 1.4/2.726, #Comment out for N_ncdm=0
        "z_pk" : 0.7, 
        "T_cmb" : 2.726} #Not necessary, but helps neatness later on

A_s = fid['A_s']
n_s = fid['n_s']
omega_b = fid['omega_b']
omega_cdm = fid['omega_cdm']
tau_reio = fid['tau_reio']
h = fid['h']
z_pk = fid['z_pk']
T_cmb = fid['T_cmb'] #Units of K

c = 2.9979e8 #Units of m/s
H = 1000. * 100. * h #Units of m/s/Mpc
kp = 0.05 * h #Units of 1/Mpc

Specify the parameter values you'd like to use to compute the forecast...

In [ ]:
#Setup redshift bins, fiducial relica masses to sample
z_table = np.arange(1.0, 1.1, 0.1)
m_chi = 0.1

# Convergence Analysis

Before running the forecast, we want to ensure our cosmological parameters are well converged about the points we are interested in using to calculate Fisher matrices. To do so, we can use the `convergence_analysis` class of `cosmicfish`. 

We pass to `convergence_analysis` some fiducial cosmology, and then it will vary the parameters of that fiducial cosmology to values specified by the user and compute the corresponding power spectrum derivatives. You can choose to pass specific values of the fiducial cosmology, or choose to vary relative to the fiducial cosmology. 

First, specify the points in parameter space you'd like to sample: 

In [ ]:
T_chi_table = np.arange(1.8, 2.1, 0.02) / T_cmb #Units of T_cmb. We specify absolute values for the T_chi parameter.
nonT_relvary = np.arange(1.01, 1.11, 0.01) #We specify values relative to the fiducial cosmology for other parameters.

Then, we generate instances of the `convergence_analysis` class. It will search for pre-existing datasets for the specified points in parameter space in the datastore directory - otherwise, it will generate the dataset and place it in datastore. 

In [ ]:
conv1 = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore); 
conv2 = cf.convergence_analysis(fid, 'omega_b', 'rel', nonT_relvary, z_table, None, classpath, datastore);
conv3 = cf.convergence_analysis(fid, 'omega_cdm', 'rel', nonT_relvary, z_table, None, classpath, datastore);
conv4 = cf.convergence_analysis(fid, 'tau_reio', 'rel', nonT_relvary, z_table, None, classpath, datastore);
conv5 = cf.convergence_analysis(fid, 'h', 'rel', nonT_relvary, z_table, None, classpath, datastore);

At this point, all the useful information generated in the convergence analysis is accessed through the methods of the `convergence_analysis` objects. We use those methods to plot the results of the analysis...

### Convergence Power Spectra

In [ ]:
conv1.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv2.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv3.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv4.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv5.plot_ps(z_index=0, xscale='linear', plotdata=True)

### Convergence Derivatives of Power Spectra

In [ ]:
conv1.plot_dps(z_index=0, xscale='linear')
conv2.plot_dps(z_index=0, xscale='linear')
conv3.plot_dps(z_index=0, xscale='linear')
conv4.plot_dps(z_index=0, xscale='linear')
conv5.plot_dps(z_index=0, xscale='linear')

### Convergence Difference of Derivatives of Power Spectra

In [ ]:
conv1.plot_delta_dps(z_index=0, xscale='linear')
conv2.plot_delta_dps(z_index=0, xscale='linear')
conv3.plot_delta_dps(z_index=0, xscale='linear')
conv4.plot_delta_dps(z_index=0, xscale='linear')
conv5.plot_delta_dps(z_index=0, xscale='linear')

### Repeat for Other $m_\chi$

We can easily repeate the analysis above for other choice of $m_\chi$. 

In [ ]:
conv1a = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., -1.0), classpath, datastore)
conv1b = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., -0.8), classpath, datastore)
conv1c = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., -0.6), classpath, datastore)
conv1d = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., -0.4), classpath, datastore)
conv1e = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., -0.2), classpath, datastore)
conv1f = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., 0.0), classpath, datastore)
conv1g = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., 0.2), classpath, datastore)
conv1h = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., 0.4), classpath, datastore)
conv1i = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., 0.6), classpath, datastore)
conv1j = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., 0.8), classpath, datastore)
conv1k = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., 1.0), classpath, datastore)

conv1a.plot_dps(z_index=1, xscale='linear')
conv1b.plot_dps(z_index=1, xscale='linear')
conv1c.plot_dps(z_index=0, xscale='linear')
conv1d.plot_dps(z_index=0, xscale='linear')
conv1e.plot_dps(z_index=0, xscale='linear')
conv1f.plot_dps(z_index=0, xscale='linear')
conv1g.plot_dps(z_index=0, xscale='linear')
conv1h.plot_dps(z_index=0, xscale='linear')
conv1i.plot_dps(z_index=0, xscale='linear')
conv1j.plot_dps(z_index=0, xscale='linear')
conv1k.plot_dps(z_index=0, xscale='linear')

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.
# Analysis - No Commentary

In [ ]:
#Import relevant packages
import os
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import cosmicfish as cf


#Instruct pyplot to use seaborn 
sns.set()

#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/test/")
datastore = cf.correct_path("~/Desktop/test/data/")
classpath = os.path.join(projectdir, "class")

#Setup fiducial cosmology, note change to N_ncdm... 
fid = {"A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "N_ncdm" : 1, #Note the change! 
        "z_pk" : 0.7, 
        "T_cmb" : 2.726,
        "T_ncdm" : 1.5/2.726} #Not necessary, but helps neatness later on

A_s = fid['A_s']
n_s = fid['n_s']
omega_b = fid['omega_b']
omega_cdm = fid['omega_cdm']
tau_reio = fid['tau_reio']
h = fid['h']
z_pk = fid['z_pk']
T_cmb = fid['T_cmb'] #Units of K

c = 2.9979e8 #Units of m/s
H = 1000. * 100. * h #Units of m/s/Mpc
kp = 0.05 * h #Units of 1/Mpc


#Setup redshift bins, fiducial relica masses, relic temps to sample
#z_table = np.arange(1.4, 1.5, 0.1)
z_table = np.array([1.4, 1.5])
m_chi = 1.0
#T_chi_table = np.arange(1.0, 2.0, 0.1) / T_cmb #Units of T_cmb
T_chi_table = np.array([1.5]) / T_cmb #Units of T_cmb
#nonT_relvary = np.arange(1.0, 1.11, 0.01)
nonT_relvary = np.array([1.0])

In [ ]:
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., 1.0)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., 0.9)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., 0.8)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., 0.7)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., 0.6)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., 0.5)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., 0.4)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., 0.3)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., 0.2)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., 0.1)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., 0.0)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., -0.1)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., -0.2)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., -0.3)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., -0.4)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., -0.5)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., -0.6)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., -0.7)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., -0.8)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., -0.9)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)

In [ ]:
datastore = cf.correct_path("~/Desktop/test/data/")
conv1 = cf.convergence_analysis(dict(fid, **{'m_ncdm' : np.power(10., -1.0)}), 'n_s', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.03)

In [ ]:
conv1 = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.01, fsky=0.3394)
#conv2 = cf.convergence_analysis(fid, 'omega_b', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.01);
#conv3 = cf.convergence_analysis(fid, 'omega_cdm', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.01);
#conv4 = cf.convergence_analysis(fid, 'tau_reio', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.01);
#conv5 = cf.convergence_analysis(fid, 'h', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.01); 

In [ ]:
conv1.plot_ps(z_index=0, xscale='linear', plotdata=True)

In [ ]:
print(conv1.spectra[0][0].class_pk[34:60]*np.power(0.70148, -3))

In [ ]:
conv1.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv2.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv3.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv4.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv5.plot_ps(z_index=0, xscale='linear', plotdata=True)

print('..................................................................................')
print('..................................................................................')
print('..................................................................................')

conv1.plot_dps(z_index=0, xscale='linear')
conv2.plot_dps(z_index=0, xscale='linear')
conv3.plot_dps(z_index=0, xscale='linear')
conv4.plot_dps(z_index=0, xscale='linear')
conv5.plot_dps(z_index=0, xscale='linear')

print('..................................................................................')
print('..................................................................................')
print('..................................................................................')

conv1.plot_delta_dps(z_index=0, xscale='linear')
conv2.plot_delta_dps(z_index=0, xscale='linear')
conv3.plot_delta_dps(z_index=0, xscale='linear')
conv4.plot_delta_dps(z_index=0, xscale='linear')
conv5.plot_delta_dps(z_index=0, xscale='linear')

In [ ]:
#Setup redshift bins, fiducial relica masses, relic temps to sample
z_table = np.arange(1.0, 1.1, 0.1)
m_chi = 0.1
T_chi_table = np.arange(1.5, 1.6, 0.1) / T_cmb #Units of T_cmb

conv1a = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.01)
conv1b = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.02)
conv1c = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.03)
conv1d = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.04)
conv1e = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.05)
conv1f = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.06)
conv1g = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.07)
conv1h = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.08)
conv1i = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.09)
conv1j = cf.convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.10)

In [ ]:
sns.set()
sns.set_palette("Blues_d", n_colors=10)
plt.figure(figsize=(15, 7.5))
ax1 = plt.subplot(1, 2, 1)

ax1.plot(conv1a.spectra[0][0].k_table, conv1a.dps[0][0], label='step=1%')
ax1.plot(conv1b.spectra[0][0].k_table, conv1b.dps[0][0], label='step=2%')
ax1.plot(conv1c.spectra[0][0].k_table, conv1c.dps[0][0], label='step=3%')
ax1.plot(conv1d.spectra[0][0].k_table, conv1d.dps[0][0], label='step=4%')
ax1.plot(conv1e.spectra[0][0].k_table, conv1e.dps[0][0], label='step=5%')
ax1.plot(conv1f.spectra[0][0].k_table, conv1f.dps[0][0], label='step=6%')
ax1.plot(conv1g.spectra[0][0].k_table, conv1g.dps[0][0], label='step=7%')
ax1.plot(conv1h.spectra[0][0].k_table, conv1h.dps[0][0], label='step=8%')
ax1.plot(conv1i.spectra[0][0].k_table, conv1i.dps[0][0], label='step=9%')
ax1.plot(conv1j.spectra[0][0].k_table, conv1j.dps[0][0], label='step=10%')

ax1.set_title(r'$\partial P_g / \partial$ T_ncdm for z=1.0, m_ncdm=0.1 [eV]')
ax1.set_xlabel(r'k [Mpc$^{-1}$]')
ax1.set_ylabel(r'[Mpc$^3$ / (units of T_ncdm)]')
ax1.legend()

ax2 = plt.subplot(1, 2, 2)
ax2.plot(conv1a.spectra[0][0].k_table, conv1a.dlogps[0][0], label='step=1%')
ax2.plot(conv1b.spectra[0][0].k_table, conv1b.dlogps[0][0], label='step=2%')
ax2.plot(conv1c.spectra[0][0].k_table, conv1c.dlogps[0][0], label='step=3%')
ax2.plot(conv1d.spectra[0][0].k_table, conv1d.dlogps[0][0], label='step=4%')
ax2.plot(conv1e.spectra[0][0].k_table, conv1e.dlogps[0][0], label='step=5%')
ax2.plot(conv1f.spectra[0][0].k_table, conv1f.dlogps[0][0], label='step=6%')
ax2.plot(conv1g.spectra[0][0].k_table, conv1g.dlogps[0][0], label='step=7%')
ax2.plot(conv1h.spectra[0][0].k_table, conv1h.dlogps[0][0], label='step=8%')
ax2.plot(conv1i.spectra[0][0].k_table, conv1i.dlogps[0][0], label='step=9%')
ax2.plot(conv1j.spectra[0][0].k_table, conv1j.dlogps[0][0], label='step=10%')
ax2.set_title(r'$\partial log(P_g) / \partial$ T_ncdm for z=1.0, m_ncdm=0.1 [eV]')
ax2.set_xlabel(r'k [Mpc$^{-1}$]')
ax2.set_ylabel(r'[Mpc$^3$ / (units of T_ncdm)]')
ax2.legend()
        
plt.show()


domega_chi_dT_chi = (3* np.power(1.5, 2.) * 0.1) / (np.power(1.95, 3.) * 94.)

sns.set()
sns.set_palette("Blues_d", n_colors=10)
plt.figure(figsize=(15, 7.5))
ax1 = plt.subplot(1, 2, 1)

ax1.plot(conv1a.spectra[0][0].k_table, conv1a.dps[0][0]/domega_chi_dT_chi, label='step=1%')
ax1.plot(conv1b.spectra[0][0].k_table, conv1b.dps[0][0]/domega_chi_dT_chi, label='step=2%')
ax1.plot(conv1c.spectra[0][0].k_table, conv1c.dps[0][0]/domega_chi_dT_chi, label='step=3%')
ax1.plot(conv1d.spectra[0][0].k_table, conv1d.dps[0][0]/domega_chi_dT_chi, label='step=4%')
ax1.plot(conv1e.spectra[0][0].k_table, conv1e.dps[0][0]/domega_chi_dT_chi, label='step=5%')
ax1.plot(conv1f.spectra[0][0].k_table, conv1f.dps[0][0]/domega_chi_dT_chi, label='step=6%')
ax1.plot(conv1g.spectra[0][0].k_table, conv1g.dps[0][0]/domega_chi_dT_chi, label='step=7%')
ax1.plot(conv1h.spectra[0][0].k_table, conv1h.dps[0][0]/domega_chi_dT_chi, label='step=8%')
ax1.plot(conv1i.spectra[0][0].k_table, conv1i.dps[0][0]/domega_chi_dT_chi, label='step=9%')
ax1.plot(conv1j.spectra[0][0].k_table, conv1j.dps[0][0]/domega_chi_dT_chi, label='step=10%')

ax1.set_title(r'$\partial P_g / \partial$ omega_ncdm for z=1.0, m_ncdm=0.1 [eV]')
ax1.set_xlabel(r'k [Mpc$^{-1}$]')
ax1.set_ylabel(r'[Mpc$^3$ / (units of omega_ncdm)]')
ax1.legend()

ax2 = plt.subplot(1, 2, 2)
ax2.plot(conv1a.spectra[0][0].k_table, conv1a.dlogps[0][0]/domega_chi_dT_chi, label='step=1%')
ax2.plot(conv1b.spectra[0][0].k_table, conv1b.dlogps[0][0]/domega_chi_dT_chi, label='step=2%')
ax2.plot(conv1c.spectra[0][0].k_table, conv1c.dlogps[0][0]/domega_chi_dT_chi, label='step=3%')
ax2.plot(conv1d.spectra[0][0].k_table, conv1d.dlogps[0][0]/domega_chi_dT_chi, label='step=4%')
ax2.plot(conv1e.spectra[0][0].k_table, conv1e.dlogps[0][0]/domega_chi_dT_chi, label='step=5%')
ax2.plot(conv1f.spectra[0][0].k_table, conv1f.dlogps[0][0]/domega_chi_dT_chi, label='step=6%')
ax2.plot(conv1g.spectra[0][0].k_table, conv1g.dlogps[0][0]/domega_chi_dT_chi, label='step=7%')
ax2.plot(conv1h.spectra[0][0].k_table, conv1h.dlogps[0][0]/domega_chi_dT_chi, label='step=8%')
ax2.plot(conv1i.spectra[0][0].k_table, conv1i.dlogps[0][0]/domega_chi_dT_chi, label='step=9%')
ax2.plot(conv1j.spectra[0][0].k_table, conv1j.dlogps[0][0]/domega_chi_dT_chi, label='step=10%')
ax2.set_title(r'$\partial log(P_g) / \partial$ omega_ncdm for z=1.0, m_ncdm=0.1 [eV]')
ax2.set_xlabel(r'k [Mpc$^{-1}$]')
ax2.set_ylabel(r'[Mpc$^3$ / (units of omega_ncdm)]')
ax2.legend()
        
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=10)
plt.figure(figsize=(15, 7.5))
ax1 = plt.subplot(1, 2, 1)

ax1.plot(conv1a.spectra[0][0].k_table, conv1a.dps[0][0]/domega_chi_dT_chi - conv1b.dps[0][0]/domega_chi_dT_chi, label='step=1%')
ax1.plot(conv1b.spectra[0][0].k_table, conv1b.dps[0][0]/domega_chi_dT_chi - conv1c.dps[0][0]/domega_chi_dT_chi, label='step=2%')
ax1.plot(conv1c.spectra[0][0].k_table, conv1c.dps[0][0]/domega_chi_dT_chi - conv1d.dps[0][0]/domega_chi_dT_chi, label='step=3%')
ax1.plot(conv1d.spectra[0][0].k_table, conv1d.dps[0][0]/domega_chi_dT_chi - conv1e.dps[0][0]/domega_chi_dT_chi, label='step=4%')
ax1.plot(conv1e.spectra[0][0].k_table, conv1e.dps[0][0]/domega_chi_dT_chi - conv1f.dps[0][0]/domega_chi_dT_chi, label='step=5%')
ax1.plot(conv1f.spectra[0][0].k_table, conv1f.dps[0][0]/domega_chi_dT_chi - conv1g.dps[0][0]/domega_chi_dT_chi, label='step=6%')
ax1.plot(conv1g.spectra[0][0].k_table, conv1g.dps[0][0]/domega_chi_dT_chi - conv1h.dps[0][0]/domega_chi_dT_chi, label='step=7%')
ax1.plot(conv1h.spectra[0][0].k_table, conv1h.dps[0][0]/domega_chi_dT_chi - conv1i.dps[0][0]/domega_chi_dT_chi, label='step=8%')

ax1.set_title(r'$\Delta(\partial P_g / \partial$ omega_ncdm) for z=1.0, m_ncdm=0.1 [eV]')
ax1.set_xlabel(r'k [Mpc$^{-1}$]')
ax1.set_ylabel(r'[Mpc$^3$ / (units of omega_ncdm)]')
ax1.legend()

ax2 = plt.subplot(1, 2, 2)
ax2.plot(conv1a.spectra[0][0].k_table, conv1a.dlogps[0][0]/domega_chi_dT_chi - conv1b.dlogps[0][0]/domega_chi_dT_chi, label='step=1%')
ax2.plot(conv1b.spectra[0][0].k_table, conv1b.dlogps[0][0]/domega_chi_dT_chi - conv1c.dlogps[0][0]/domega_chi_dT_chi, label='step=2%')
ax2.plot(conv1c.spectra[0][0].k_table, conv1c.dlogps[0][0]/domega_chi_dT_chi - conv1d.dlogps[0][0]/domega_chi_dT_chi, label='step=3%')
ax2.plot(conv1d.spectra[0][0].k_table, conv1d.dlogps[0][0]/domega_chi_dT_chi - conv1e.dlogps[0][0]/domega_chi_dT_chi, label='step=4%')
ax2.plot(conv1e.spectra[0][0].k_table, conv1e.dlogps[0][0]/domega_chi_dT_chi - conv1f.dlogps[0][0]/domega_chi_dT_chi, label='step=5%')
ax2.plot(conv1f.spectra[0][0].k_table, conv1f.dlogps[0][0]/domega_chi_dT_chi - conv1g.dlogps[0][0]/domega_chi_dT_chi, label='step=6%')
ax2.plot(conv1g.spectra[0][0].k_table, conv1g.dlogps[0][0]/domega_chi_dT_chi - conv1h.dlogps[0][0]/domega_chi_dT_chi, label='step=7%')
ax2.plot(conv1h.spectra[0][0].k_table, conv1h.dlogps[0][0]/domega_chi_dT_chi - conv1i.dlogps[0][0]/domega_chi_dT_chi, label='step=8%')
ax2.plot(conv1i.spectra[0][0].k_table, conv1i.dlogps[0][0]/domega_chi_dT_chi - conv1j.dlogps[0][0]/domega_chi_dT_chi, label='step=9%')
ax2.set_title(r'$\Delta(\partial log(P_g) / \partial$ omega_ncdm) for z=1.0, m_ncdm=0.1 [eV]')
ax2.set_xlabel(r'k [Mpc$^{-1}$]')
ax2.set_ylabel(r'[Mpc$^3$ / (units of omega_ncdm)]')
ax2.legend()
        
plt.show()




.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

In [ ]:
#Import relevant packages
import os
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import cosmicfish as cf


#Instruct pyplot to use seaborn 
sns.set()

#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/test/")
datastore = cf.correct_path("~/Desktop/test/data/")
classpath = os.path.join(projectdir, "class")

#Setup fiducial cosmology, note change to N_ncdm... 
fid = {"A_s" : 2.2321e-9, 
       "n_s" : 0.96659,
       "omega_b" : 0.02226,
       "omega_cdm" : 0.11271,
       "tau_reio" : 0.059888,
       "h" : 0.70148,
       "z_pk" : 1.4, 
       "T_cmb" : 2.726,
       "N_ncdm" : 3., 
       #"T_ncdm" : 1.5 / 2.726, #Only uncomment when forecasting relics
       "m_ncdm" : 0.02} 


c = 2.9979e8 #Units of m/s
H = 1000. * 100. * fid['h'] #Units of m/s/Mpc
kp = 0.05 * fid['h'] #Units of 1/Mpc\
skycover = 14000. #Sky coverage of survey in degrees^2


#Setup redshift bins, fiducial relica masses, relic temps to sample
z_table = np.arange(0.65, 1.85, 0.1)
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
mu_step = 0.05 #For calculating numerical integral wrt mu between -1 and 1 

In [ ]:
lightrelic = cf.forecast(classpath, datastore, 'neutrino', fid, z_table, dNdz, fcoverage_deg=skycover, dstep=0.03)

In [ ]:
lightrelic.gen_pg()

In [ ]:
lightrelic.print_v_table()

In [ ]:
lightrelic.gen_fisher(mu_step)

In [ ]:
lightrelic.print_P_table()

In [ ]:
lssfisher = lightrelic.fisher
lssfisher = pd.DataFrame(lssfisher,  columns=['omega_b', 'omega_cdm', 'n_s', 'A_s', 'tau_reio', 'h', 'M_ncdm', 'sigma_fog', 'bLbar', 'alpha_k2'])
lssfisher.iloc[:,7] *= 1e3 #To correct units on sigma_fog
lssfisher.iloc[7,:] *= 1e3 #To correct units on sigam_fog
lssfisher

In [ ]:
lssfisher.iloc[:,6] *= 3. # Correct from total neutrino mass to single neutrino mass
lssfisher.iloc[6,:] *= 3. # Correct from total neutrino mass to single neutrino mass
lssfisher = lssfisher.rename(index=str, columns={"M_ncdm": "m_ncdm"})
lssfisher

In [ ]:
lsscovariance = np.linalg.inv(lssfisher)
lsscovariance = pd.DataFrame(lsscovariance, columns=lssfisher.columns)
lsscovariance

In [ ]:
cmbfisher = pd.read_csv("~/Desktop/CMBS4_Fisher.dat", sep='\t', header=0)
pd.DataFrame(cmbfisher)

In [ ]:
cmbfisher.iloc[:,5] *= 100.
cmbfisher.iloc[5,:] *= 100.
cmbfisher.iloc[:,6] *= 3.
cmbfisher.iloc[6,:] *= 3.
cmbfisher = cmbfisher.rename(index=str, columns={"H_0": "h", "M_ncdm": "m_ncdm"})
cmbfisher

In [ ]:
cmbcovariance = pd.DataFrame(np.linalg.inv(cmbfisher), columns=cmbfisher.columns)
cmbcovariance

In [ ]:
fullfisher = lssfisher 
fullfisher.iloc[0:6, 0:6] += cmbfisher
fullfisher

In [ ]:
fullcovariance = pd.DataFrame(np.linalg.inv(fullfisher), columns=fullfisher.columns)
fullcovariance

In [ ]:
print("Fully Marginalized Error on A_s: ", np.sqrt(fullcovariance.iloc[0,0]))
print("Fully Marginalized Error on n_s: ", np.sqrt(fullcovariance.iloc[1,1]))
print("Fully Marginalized Error on omega_b: ", np.sqrt(fullcovariance.iloc[2,2]))
print("Fully Marginalized Error on omega_cdm: ", np.sqrt(fullcovariance.iloc[3,3]))
print("Fully Marginalized Error on h: ", np.sqrt(fullcovariance.iloc[4,4]))
print("Fully Marginalized Error on tau_reio: ", np.sqrt(fullcovariance.iloc[5,5]))
print("Fully Marginalized Error on M_ncdm: ", np.sqrt(fullcovariance.iloc[6,6]))
print("Fully Marginalized Error on sigma_fog: ", np.sqrt(fullcovariance.iloc[7,7]))
print("Fully Marginalized Error on bLbar: ", np.sqrt(fullcovariance.iloc[8,8]))
print("Fully Marginalized Error on alpha_k2: ", np.sqrt(fullcovariance.iloc[9, 9]))

In [ ]:
pd.DataFrame(fullcovariance).to_csv("~/Desktop/inv_fullfisher.mat", sep="\t", index=False, header=['# omega_b', 'omega_cdm', 'n_s', 'A_s', 'tau_reio', 'h', 'm_0', 'sigma_fog', 'bLbar', 'alpha_k2'])
#pd.DataFrame(fullcovariance).to_csv("~/Desktop/inv_fullfisher.mat", sep="\t", index=False, header=['# omega_b', 'omega_cdm', 'n_s', 'A_s', 'tau_reio', 'h', 'm_0', 'sigma_fog'])

In [ ]:
pd.DataFrame(cmbcovariance).to_csv("~/Desktop/inv_cmbfisher.mat", sep="\t", index=False, header=['# omega_b', 'omega_cdm', 'n_s', 'A_s', 'tau_reio', 'h', 'm_0'])

In [ ]:
pd.DataFrame(lsscovariance).to_csv("~/Desktop/inv_lssfisher.mat", sep="\t", index=False, header=['# omega_b', 'omega_cdm', 'n_s', 'A_s', 'tau_reio', 'h', 'm_0', 'sigma_fog', 'bLbar', 'alpha_k2'])
#pd.DataFrame(lsscovariance).to_csv("~/Desktop/inv_lssfisher.mat", sep="\t", index=False, header=['# omega_b', 'omega_cdm', 'n_s', 'A_s', 'tau_reio', 'h', 'm_0', 'sigma_fog'])

In [ ]:
print("Looking for k = ", 0.2 * fid['h'], " [Mpc^-1]")
print(lightrelic.k_table)
kidx = 69
muidx = 20

print(lightrelic.k_table[kidx])
print(np.arange(-1., 1., mu_step)[muidx])

for zidx, zval in enumerate(z_table): 
    print("For z = {0:.2f},\t P(0.2h, 0) = {1:.2f},\t nbar * P_(0.2h, 0) = {1:.2e}".format(zval, 
                                                                                         lightrelic.Pm[zidx][kidx][muidx],
                                                                                         lightrelic.Pm[zidx][kidx][muidx]*lightrelic.n_densities[zidx]))

In [ ]:
#For each z, plot inferred spectrum vs. CLASS data
sns.set()
sns.set_palette("Blues_d", n_colors=2*len(z_table)+1)
plt.figure(figsize=(15, 7.5))
for zidx, zval in enumerate(z_table[0:1]): 
    ps = lightrelic.spectra_mid[zidx]
    plt.plot(ps.k_table, ps.ps_table, label=("z = " + str(zval)))
    plt.plot(ps.class_pk['k (h/Mpc)']*0.70148,
             ps.class_pk['P (Mpc/h)^3']*np.power(0.70148, -3),
             label='CLASS P(k) Data',
             marker='x',
             linestyle=':')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$P_g$ [Mpc$^3$]')
plt.xlim(-0.01, 0.2)
plt.legend()
plt.show()
#Also plot difference between the two 

In [ ]:
len(['a','b'])